# huggingface/Transformerを用いた事前学習済み日本語BERTモデルによるテキスト分類

今回は、[huggingface/Transformer](https://huggingface.co/transformers/)で提供されている[日本語BERT](https://github.com/cl-tohoku/bert-japanese)を用いて日本語のAmazonの商品レビューに対する感情分析を行います。 
つまり、そのレビューが Positive (Rating が 5 or 4) か、Negative (Rating が 1 or 2)なのかを判定します。 これは文書を Positive か Negative に分類する2クラスの分類問題として扱うことができます。

## **データの取得**

Amazon の商品レビューデータセットは [Registry of Open Data on AWS](https://registry.opendata.aws/) で公開されており、 
以下からダウンロード可能です。
このノートブックでは、日本語のデータセットをダウンロードします。
- データセットの概要  
https://registry.opendata.aws/amazon-reviews/

- 日本語のデータセット(readme.htmlからたどることができます）  
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

以下では、データをダウンロードして解凍 (unzip) します。

In [ ]:
import urllib.request
import os
import gzip
import shutil

download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

## **データの前処理**

ダウンロードしたデータには学習に不要なデータや直接利用できないデータもあります。以下の前処理で利用できるようにします。

1. ダウンロードしたデータには不要なデータも含まれているので削除し、2クラス分類 (positive が 1, negative が 0)となるように評価データを加工します。
2. 学習データ、検証データに分けて、保存します。

### **データの加工**

今回利用しないデータは以下の４つです。必要なデータだけ選んで保存します。
- 評価データ `star_rating` と レビューのテキストデータ `review_body` 以外のデータ
- 評価が 3 のデータ (positive でも negative でもないデータ)
- 日本語以外のデータ
- `review_body`が空白のデータ

また、評価が1, 2 のデータはラベル 0 (negative) に、評価が4, 5 のデータはラベル 1 (positive) にします。

In [ ]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv(tsv_file_path, sep ='\t')
df_pos_neg = df.loc[:, ["star_rating", "review_body"]]
df_pos_neg = df_pos_neg[df_pos_neg.star_rating != 3]
df_pos_neg.loc[df_pos_neg.star_rating < 3, "star_rating"] = 0
df_pos_neg.loc[df_pos_neg.star_rating > 3, "star_rating"] = 1

def is_japanese(s):
    return True if re.search(r'[ぁ-んァ-ン]', s) else False

def is_almost_blank(s):
    return True if len(s.strip().split('\t')) == 2 else False

def func_to_row(x):
    if not is_japanese(x["review_body"]):
        x["review_body"] = np.nan
    
    elif not is_almost_blank(x["review_body"]):
        x["review_body"] = np.nan
    
    return x

labeled_df = df_pos_neg.apply(lambda x: func_to_row(x), axis =1)
labeled_df.dropna()

### **データの分割**

学習には学習データと検証データが必要なため、それぞれ準備します。
`train_ratio` で設定した割合のデータを学習データとし、残ったデータを検証データに分割して利用します。

In [ ]:
import numpy as np
import os
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

# Swap positions of "review_body","star_rating" because transform.py requires this order.
labeled_df = df_pos_neg.loc[:, ["review_body","star_rating"]]
data_size = len(labeled_df.index)
train_ratio = 0.9
train_index = np.random.choice(data_size, int(data_size*train_ratio), replace=False)
valid_index = np.setdiff1d(np.arange(data_size), train_index)

np.savetxt('train.tsv',labeled_df.iloc[train_index].values, fmt="%s\t%i") 
np.savetxt('valid.tsv',labeled_df.iloc[valid_index].values, fmt="%s\t%i") 

print("Data is splitted into:")
print("Training data: {} records.".format(len(train_index)))
print("valid data: {} records.".format(len(valid_index)))

### **ヘッダの追加**
解析するテキストを`review_body`、ポジティブかネガティブかを表すラベルを`star_rating`としてヘッダを与えます。

In [ ]:
!sed -i '1s/^/review_body\tstar_rating\n/' train.tsv
!sed -i '1s/^/review_body\tstar_rating\n/' valid.tsv

## **学習の実行**
### **学習データのアップロード**
S3に学習データと検証データをアップロードします。

In [ ]:
import sagemaker

sess = sagemaker.Session()

s3_train_data = sess.upload_data(path='train.tsv', key_prefix='amazon-review-data-tsv')
s3_valid_data = sess.upload_data(path='valid.tsv', key_prefix='amazon-review-data-tsv')
print("Training data is uploaded to {}".format(s3_train_data))
print("Training data is uploaded to {}".format(s3_valid_data))

data_channels = {'train': s3_train_data, 'val': s3_valid_data}

### **学習ジョブの実行**
今回はGPUインスタンスであるml.p3.2xlargeを使用します。テストのため、epochは1とします。
`src`ディレクトリにある`requirements.txt`、`train_and_deploy.py`、`EarlyStopping.py`が
学習インスタンスへアップロードされます。

In [ ]:
from sagemaker.pytorch import PyTorch

role = sagemaker.get_execution_role()

estimator = PyTorch(entry_point='train_and_deploy.py',
                    source_dir = 'src',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=2,
                    train_instance_type='ml.p3.2xlarge',
                    hyperparameters={
                        'epochs': 1,
                    })

In [ ]:
estimator.fit(data_channels)

## **デプロイ**
学習したモデルをデプロイします。推論はCPUインスタンスで行うことができます。<br>
最後に`!`記号が現れたらデプロイ完了です。

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

## **推論**
実際の文を用いた推論を行います。
response_hypoの値が1に近ければポジティブ、0に近ければネガティブです。

Pytorchコンテナではserializer等がnumpyであるため、deploy後に以下のようにしてserializerを変更します。

In [ ]:
from sagemaker.predictor import json_serializer
from sagemaker.predictor import json_deserializer
predictor.content_type = 'application/json'
predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

In [ ]:
response_hypo = predictor.predict("価格にもクオリティにも満足した。")
prediction_label = "ポジティブ" if response_hypo[0] >= 0.5 else "ネガティブ"
print("label: {} (hypo value: {})".format(prediction_label,response_hypo[0]))